In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
from google.colab import files
import pickle
import os
from scipy import stats
from scipy.stats import zscore

!pip install missingno
import missingno as msno

!pip install fancyimpute
import fancyimpute

!pip install --upgrade autoviz
from autoviz.AutoViz_Class import AutoViz_Class

warnings.filterwarnings("ignore")
%matplotlib inline

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=5149779bace07a4400bdd380e481030994826737cf01d1ffba2778aadd34e813
  Stored in directory: /root/.cache/pip/wheels/df/20/91/e4850b9a31cf660c1bc95515d3bcbc8010e869e5de6d5baf07
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=b4bfcb35525c6f4223c73bb119d8fc2e0edc78f9a8f03ad2bd89f6bd98a1f339
  Stored in directory: /root/.cache/pip/wheels/89/11/61/aa5a3167bfff49218cd043a73a83034b9eadd858b0059521be
Successfully built fancyimpute knnimpute
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
np.random.seed(31071967)

In [ ]:
download_pickels = True

In [ ]:
import os, pathlib, sys
!mkdir -p pickles
pickle_file = '/content/pickles/load_default_feature_eng.pkl'

if not os.path.exists(pickle_file):
# Upload file
  uploaded = files.upload()

# Read pickle into DataFrame
df = pd.read_pickle(pickle_file)

df.head(10)

id     loan_amnt  funded_amnt_inv  installment  annual_inc  \
0   545583     2500          2500           85.0       20004.0    
1   532101     5000          5000          176.0       59000.0    
2   877788     7000          7000          226.0       53796.0    
3   875406     2000          2000           60.0       30000.0    
4   506439     3600          3600          117.0      675048.0    
5   981465     8000          8000          243.0       77736.0    
6   749050     6000          6000          187.0       35000.0    
7  1016373    25600         25473          599.0       86000.0    
8   786870    19750         19750          559.0       72500.0    
9  1019261     6250          6250          224.0       28000.0    

   verification_status  purpose  addr_state   dti   inq_last_6mths  open_acc  \
0           0              9         22      19.86        5.0          7.0     
1           0              2         33      19.57        1.0          7.0     
2           0              9         42      10.80        3.0          7.0     
3           0              2         33       3.60        0.0          7.0     
4           0              9          1       1.55        4.0          8.0     
5           2              9          3       6.07        0.0         12.0     
6           0              2         24      13.13        0.0          5.0     
7           2              2         33      26.51        1.0         16.0     
8           2              2         24      19.96        0.0         15.0     
9           2              9         34      13.76        0.0          2.0     

   pub_rec  revol_bal  revol_util  total_acc  total_pymnt  issue_d_month  \
0    0.0       981.0      21.30      10.0        3075.0         7.0        
1    0.0     18773.0      99.90      15.0        2949.0         6.0        
2    0.0      3269.0      47.20      20.0        8082.0         9.0        
3    0.0         0.0       0.00      15.0        2162.0         9.0        
4    0.0         0.0       0.00      25.0        4206.0         4.0        
5    0.0      4182.0      13.60      49.0        8725.0        10.0        
6    0.0      5864.0      47.70       9.0        6718.0         5.0        
7    0.0     33021.0      70.80      32.0       32840.0        11.0        
8    0.0     21544.0      98.70      44.0       27545.0         6.0        
9    1.0         0.0       8.46      15.0        6689.0        11.0        

   earliest_cr_line_year  earliest_cr_line_month  last_pymnt_d_year  \
0         2005.0                    8.0                2013.0         
1         1994.0                    4.0                2011.0         
2         1998.0                    3.0                2014.0         
3         1975.0                    1.0                2014.0         
4         1998.0                    4.0                2013.0         
5         1996.0                    7.0                2014.0         
6         2003.0                   10.0                2014.0         
7         1983.0                   10.0                2014.0         
8         1992.0                    3.0                2013.0         
9         1999.0                   10.0                2014.0         

   last_pymnt_d_month  last_credit_pull_d_year  last_credit_pull_d_month  \
0          7.0                 2016.0                      6.0             
1         11.0                 2012.0                      3.0             
2          3.0                 2014.0                      3.0             
3          2.0                 2016.0                      6.0             
4          5.0                 2016.0                      6.0             
5          4.0                 2014.0                      4.0             
6          5.0                 2014.0                      5.0             
7          4.0                 2016.0                      6.0             
8          6.0                 2016.0                      2.0             
9      

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
TARGET = 'repay_fail'
def to_numeric_numpy(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # א. המרה של Int64/Float64 (nullable) ל-float32 – בטוח גם אם אין NaN
    ext = X.select_dtypes(include=['Int64', 'Float64']).columns
    X[ext] = X[ext].astype('float32')
    # ב. float64 => float32 (חיסכון בזיכרון ומהירות)
    f64 = X.select_dtypes(include=['float64']).columns
    X[f64] = X[f64].astype('float32')
    # ג. int8/int64 וכו' — להשאיר כ-int (או להפוך ל-int32 אם רוצים אחידות)
    # i32 = X.select_dtypes(include=['int64']).columns
    # X[i32] = X[i32].astype('int32')
    return X

X = df.drop(columns=[TARGET])
y = df[TARGET]

X = to_numeric_numpy(X)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38478 entries, 0 to 38477
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             38478 non-null  int64  
 1   loan_amnt                      38478 non-null  Int64  
 2   funded_amnt_inv                38478 non-null  Int64  
 3   installment                    38478 non-null  float64
 4   annual_inc                     38478 non-null  float64
 5   verification_status            38478 non-null  int8   
 6   purpose                        38478 non-null  int8   
 7   addr_state                     38478 non-null  int8   
 8   dti                            38478 non-null  float64
 9   inq_last_6mths                 38478 non-null  float64
 10  open_acc                       38478 non-null  float64
 11  pub_rec                        38478 non-null  float64
 12  revol_bal                      38478 non-null 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('repay_fail',axis=1), df['repay_fail'], test_size=0.3, random_state=42)

In [ ]:
# Define the classification metrics function
def classificationMetrics(y, yhat):
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
    return metrics

In [ ]:
from xgboost import XGBClassifier
model =  XGBClassifier()

# Convert the entire dataframes to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))
classificationMetrics(y_test, y_pred)

{'Accuracy': 0.9807692307692307,
 'Precision': 0.9709517923362175,
 'Recall': 0.8997709049255441,
 'F1-Score': 0.9340071343638525}

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier

In [ ]:
techniques = {
    "ROS": RandomOverSampler(random_state=47),
    "RUS": RandomUnderSampler(random_state=47),
    "SMOTE": SMOTE(random_state=47),
    "SMOTETomek": SMOTETomek(random_state=47)
}

In [ ]:
# Evaluate and collect results using list comprehension
results = [
    {
        "Technique": name,
        "Accuracy": accuracy_score(y_test, LogisticRegression(random_state=42).fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "Precision": precision_score(y_test, LogisticRegression(random_state=42).fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "Recall": recall_score(y_test, LogisticRegression(random_state=42).fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32'))),
        "F1-Score": f1_score(y_test, LogisticRegression(random_state=42).fit(*technique.fit_resample(X_train.astype('float32'), y_train)).predict(X_test.astype('float32')))
    }
    for name, technique in techniques.items()
]
# Display results
results_df = pd.DataFrame(results)
print(results_df)

   Technique   Accuracy  Precision   Recall   F1-Score
0         ROS  0.934598  0.753972   0.842497  0.795780
1         RUS  0.934252  0.757165   0.832188  0.792906
2       SMOTE  0.938236  0.776647   0.830470  0.802657
3  SMOTETomek  0.936244  0.776258   0.812715  0.794068


In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df

Technique   Accuracy  Precision   Recall   F1-Score
0         ROS  0.934598  0.753972   0.842497  0.795780
1         RUS  0.934252  0.757165   0.832188  0.792906
2       SMOTE  0.938236  0.776647   0.830470  0.802657
3  SMOTETomek  0.936244  0.776258   0.812715  0.794068

Before balancing you obtained:

Accuracy ≈ 0.981 | Precision ≈ 0.971 | Recall ≈ 0.900 | F1 ≈ 0.934

After balancing (all techniques):

Accuracy ≈ 0.934–0.938 | Precision ≈ 0.754–0.777 | Recall ≈ 0.813–0.842 | F1 ≈ 0.793–0.803

Meaning: all the known metrics deteriorated (including Recall), therefore — based on the results you presented — there is no reason to choose any of the balancing techniques.


כל המדדים הידועים הידרדרו (כולל Recall), לכן במצב הנתונים  – אין סיבה לבחור באף אחת מטכניקות האיזון.